## Scrape Kaggle Rankings 

This notebook is used to scrape the following from Kaggle
- Ranking 
- Name of Company
- Job Title 

In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import time

import pandas as pd
import matplotlib.pyplot as plt

Start with grabbing the URLs for the top 1,000 Kaggle Competitors 

In [2]:
# base url to scrape 
url = 'https://www.kaggle.com/rankings?group=competitions&page=1&pageSize=20'

In [3]:
# make driver element 
driver = webdriver.Chrome()
driver.get(url)

In [34]:
# grab html 
my_html = driver.page_source
soup = BeautifulSoup(my_html, 'html.parser')
kagglers = soup.findAll('div', 'leaderboards__item-wrapper null')

In [35]:
len(kagglers)

1020

In [36]:
kaggler_rankings = []
for counter, kag in enumerate(kagglers):
    rank = int(kag.find('div','leaderboards__rank').text)
    tier = kag.find('div','leaderboards__tier').find('img')['title']
    url = kag.find('div','leaderboards__avatar').find('a')['href']
    kag_res = {'rank': rank, 'tier': tier, 'url': url}
    kaggler_rankings.append(kag_res)
kaggle_df = pd.DataFrame(kaggler_rankings)

In [38]:
kaggle_df.head()

,rank,tier,url
0,1,grandmaster,/philippsinger
1,2,grandmaster,/wowfattie
2,3,grandmaster,/christofhenkel
3,4,grandmaster,/bestfitting
4,5,grandmaster,/dott1718


Now that I have the URLs, I can get the job title and company they work at from their specific Kaggle page 

In [39]:
# add new columns to dataframe
kaggle_df['job'] = ''
kaggle_df['location'] = ''

In [ ]:
driver_user = webdriver.Chrome()
for counter, row in kaggle_df.iterrows():
    if (counter > 0) & (counter % 10 == 0) & (counter % 100 != 0):
        driver_user.close()
        driver_user = webdriver.Chrome()
        time.sleep(2)
    elif (counter > 0) & (counter % 100 == 0):
        print(counter)
        driver_user.close()
        time.sleep(900)
        driver_user = webdriver.Chrome()
    user_url = 'https://www.kaggle.com' + row['url']
    driver_user.get(user_url)
    time.sleep(2)
    html = driver_user.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
    time.sleep(2)
    soup_user = BeautifulSoup(html, 'html.parser')
    try:
        c_j = soup_user.find('p', 'profile__user-occupation').text
    except AttributeError:
        c_j = 'unknown'
    try:
        loc = soup_user.find('p', 'profile__user-location').text
    except AttributeError:
        loc = 'unknown'
    kaggle_df.loc[counter, 'job'] = c_j
    kaggle_df.loc[counter, 'location'] = loc

In [165]:
# save to computer
kaggle_df.to_csv('kaggle_data.csv', index = False)

In [169]:
# clean job 
title = []
company = []

for j in kaggle_df['job']:
    split_on_at = j.split(' at ')
    if len(split_on_at) == 2:
        title.append(split_on_at[0].strip())
        company.append(split_on_at[1].strip())
    else:
        title.append('unknown')
        company.append('unknown')

In [170]:
kaggle_df['title'] = title
kaggle_df['company'] = company

In [181]:
# save to computer
kaggle_df.to_csv('kaggle_data.csv', index = False)